In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd
import statistics as st

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [85]:
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
print('x' in np.arange(5))   #returns False, without Warning

False


In [3]:
boston = datasets.load_boston() 

In [4]:
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target)

In [5]:
## Falta implementar Cross Validation 

In [6]:
X= pd.DataFrame(data=boston.data, columns = boston['feature_names']) 
    
Y = pd.DataFrame(data=boston.target)
    
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [78]:
col_select_train= pd.DataFrame()




In [82]:
len(X_train)

379

In [80]:
col_select_train

""


In [83]:
if len(X_train) == 0:
    print('ok')

## Forward Selection 

In [134]:
def forward_selection(X=boston.data,y=boston.target,tol=0.001):
    '''
    Elije la mejor variable, una a la vez, para un modelo lineal, y va agregando variables. 
    Devuelve las columnas seleccionadas con sus datos, para el train y el test 
    '''
    
    X= pd.DataFrame(data=boston.data, columns = boston['feature_names']) 
    
    Y = pd.DataFrame(data=boston.target)
    
    X_train, X_test, y_train, y_test = train_test_split(X,Y,random_state=15)
    
    ## Convertirlos en panda data frame: 
    
    
    col_select_train= pd.DataFrame()
    col_select_test = pd.DataFrame()
    max_R_2 = 0.1
    max_R_2_ant = 0
    
    
    while np.abs(max_R_2 - max_R_2_ant) > tol and X_train.shape[1] > 0:         
        
        if len(col_select_train) == 0:  # Primera vuelta, crea el data frame para la primera variable 
            for i in range (X_train.shape[1]):
                R_cuadrado= []
                X_train_m= X_train.iloc[:,i][:, np.newaxis]
                X_test_m= X_test.iloc[:,i][:, np.newaxis]
                
                modelo=LinearRegression()
                modelo.fit(X_train_m, y_train)
                
                #Voy agregando todos los R_cuadrados
                R_cuadrado.append(modelo.score(X_test_m, y_test) ) 
                
            #Me quedo con el mejor R_Cuadrado: 
            max_R_2_ant = max_R_2
            max_R_2 = max(R_cuadrado)
            max_index = R_cuadrado.index(max_R_2)
                
            #Agrego la columna perteneciente al mejor R cuadrado a mis variables seleccionadas:
            col_select_train= pd.DataFrame(data=X_train.iloc[:,max_index])
            col_select_test= pd.DataFrame(data=X_test.iloc[:,max_index])
            
            
            #Eliminar la columna de X_train y X_test
            X_train.drop(X_train.columns[max_index],axis=1,inplace=True)
            X_test.drop(X_test.columns[max_index],axis=1,inplace=True)
            
            
        else: 
            for i in range (X_train.shape[1]):
                R_cuadrado= []
                
                ## Panda data frame, con column_select_train + la nueva variable a explorar: 
                X_train_m= pd.concat([col_select_train, X_train.iloc[:,i] ], axis=1)
                X_test_m= pd.concat([col_select_test, X_test.iloc[:,i] ], axis=1)

                modelo=LinearRegression()
                modelo.fit(X_train_m, y_train)
                
                assert sum(X_train_m['CRIM'] == 'Nan') == 0
                
                #print(X_train_m)
                #print(X_train_m.shape)
                #print('estamos en la i:', i)
                #print('coef', modelo.coef_)
                #print('score', modelo.score(X_test_m, y_test))
                
                #Voy agregando todos los R_cuadrados
                R_cuadrado.append(modelo.score(X_test_m, y_test) ) 
                
            #Me quedo con el mejor R_Cuadrado: 
            max_R_2_ant = max_R_2
            max_R_2 = max(R_cuadrado)
            print('R cuadrado es de: ', max_R_2)
            max_index = R_cuadrado.index(max_R_2)

            #Agrego la columna perteneciente al mejor R cuadrado a mis variables seleccionadas:
            col_select_train= pd.concat([col_select_train, X_train.iloc[:,max_index] ], axis=1)
            col_select_test= pd.concat([col_select_test, X_test.iloc[:,max_index] ], axis=1)

            #col_select_train.append(X_train[:,max_index])
            #col_select_test.append(X_test[:,max_index])

            ## Elimino la columna elegida de X_train y X_test para que no pueda volver a ser elegida
            X_train.drop(X_train.columns[max_index],axis=1,inplace=True)
            X_test.drop(X_test.columns[max_index],axis=1,inplace=True)

    
    return col_select_train, col_select_test


## Luego hay que hacer un CORTE, de tolerancia, de cuanto tiene que ser la diferencia (seria un WHILE, tol <...)


In [147]:
col_select_train_0, col_select_test = forward_selection(tol=0.000001)

R cuadrado es de:  0.5337761885389227
R cuadrado es de:  0.5301238007421429
R cuadrado es de:  0.5296154294453219
R cuadrado es de:  0.5581579077673682
R cuadrado es de:  0.564492836339777
R cuadrado es de:  0.6328006357251179
R cuadrado es de:  0.6328597105363207
R cuadrado es de:  0.6524690541538684
R cuadrado es de:  0.6517928418858312
R cuadrado es de:  0.6562463910313383
R cuadrado es de:  0.6801828387233039
R cuadrado es de:  0.6850106699596988


In [148]:
col_select_train_0

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
163,1.51902,0.0,19.58,1.0,0.605,8.375,93.9,2.1620,5.0,403.0,14.7,388.45,3.32
228,0.29819,0.0,6.20,0.0,0.504,7.686,17.0,3.3751,8.0,307.0,17.4,377.51,3.92
84,0.05059,0.0,4.49,0.0,0.449,6.389,48.0,4.7794,3.0,247.0,18.5,396.90,9.62
189,0.08370,45.0,3.44,0.0,0.437,7.185,38.9,4.5667,5.0,398.0,15.2,396.90,5.39
322,0.35114,0.0,7.38,0.0,0.493,6.041,49.9,4.7211,5.0,287.0,19.6,396.90,7.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,19.60910,0.0,18.10,0.0,0.671,7.313,97.9,1.3163,24.0,666.0,20.2,396.90,13.44
133,0.32982,0.0,21.89,0.0,0.624,5.822,95.4,2.4699,4.0,437.0,21.2,388.69,15.03
396,5.87205,0.0,18.10,0.0,0.693,6.405,96.0,1.6768,24.0,666.0,20.2,396.90,19.37
245,0.19133,22.0,5.86,0.0,0.431,5.605,70.2,7.9549,7.0,330.0,19.1,389.13,18.46


In [138]:
col_select_train, col_select_test = forward_selection(tol=0.0001)

R cuadrado es de:  0.5337761885389227
R cuadrado es de:  0.5301238007421429
R cuadrado es de:  0.5296154294453219
R cuadrado es de:  0.5581579077673682
R cuadrado es de:  0.564492836339777
R cuadrado es de:  0.6328006357251179
R cuadrado es de:  0.6328597105363207


In [141]:
col_select_train

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS
163,1.51902,0.0,19.58,1.0,0.605,8.375,93.9,2.1620
228,0.29819,0.0,6.20,0.0,0.504,7.686,17.0,3.3751
84,0.05059,0.0,4.49,0.0,0.449,6.389,48.0,4.7794
189,0.08370,45.0,3.44,0.0,0.437,7.185,38.9,4.5667
322,0.35114,0.0,7.38,0.0,0.493,6.041,49.9,4.7211
...,...,...,...,...,...,...,...,...
375,19.60910,0.0,18.10,0.0,0.671,7.313,97.9,1.3163
133,0.32982,0.0,21.89,0.0,0.624,5.822,95.4,2.4699
396,5.87205,0.0,18.10,0.0,0.693,6.405,96.0,1.6768
245,0.19133,22.0,5.86,0.0,0.431,5.605,70.2,7.9549


In [144]:
col_select_train_2, col_select_test = forward_selection(tol=0.001)

R cuadrado es de:  0.5337761885389227
R cuadrado es de:  0.5301238007421429
R cuadrado es de:  0.5296154294453219


In [145]:
col_select_train_2

,CRIM,ZN,INDUS,CHAS
163,1.51902,0.0,19.58,1.0
228,0.29819,0.0,6.20,0.0
84,0.05059,0.0,4.49,0.0
189,0.08370,45.0,3.44,0.0
322,0.35114,0.0,7.38,0.0
...,...,...,...,...
375,19.60910,0.0,18.10,0.0
133,0.32982,0.0,21.89,0.0
396,5.87205,0.0,18.10,0.0
245,0.19133,22.0,5.86,0.0


In [142]:
col_select_train_3, col_select_test_2 = forward_selection(tol=0.09)

R cuadrado es de:  0.5337761885389227


In [146]:
col_select_train_3

,CRIM,ZN
163,1.51902,0.0
228,0.29819,0.0
84,0.05059,0.0
189,0.08370,45.0
322,0.35114,0.0
...,...,...
375,19.60910,0.0
133,0.32982,0.0
396,5.87205,0.0
245,0.19133,22.0
